# EnergyPlus and TRNSYS Example

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

# There are 2 environments in this example: a TRNSYS-based one and an EnergyPlus-based one.
sim.create_meta_model("MetaTrnsys", [("Treturn", "Cdeg")], [("Tsupply", "Cdeg")]) # Set, Get
sim.create_environment("EnvTrnsys", "wrapper_trnsys.py", "Dockerfile_na")

sim.create_meta_model("MetaEplus", [("Ts_second", "Cdeg")], [("Tr_second", "Cdeg")]) # Set, Get
sim.create_environment("EnvEplus", "wrapper_eplus.py", "Dockerfile_na")

# There is one model per each environment in this example. 
# OBS!! The name Base# indicates a model. Each model is contained in a node. There can be more nodes in each environment.
sim.add_node("Base0", "MetaTrnsys", "EnvTrnsys", init_values={"Treturn":40}, files=["COG_waste.fmu"],local=True)
sim.add_node("Base1", "MetaEplus", "EnvEplus", init_values={}, files=["Residential_DH.fmu"],local=True)

# Links among nodes
sim.add_link("Base0", "Tsupply", "Base1", "Ts_second") # get from Base0 and set to Base1
sim.add_link("Base1", "Tr_second", "Base0", "Treturn") # get from Base1 and set to Base0

# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.
sim.create_sequence([["Base0"], ["Base1"]])
sim.set_time_unit("seconds")
sim.create_steps([3600] * 24)

Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [3]:
sim.run()

INFO :: Waiting for local nodes to run...
INFO :: Simulation finished in 0 min and 49 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,OUT,Base0,Tsupply
1,OUT,Base1,Tr_second
2,IN,Base0,Treturn
3,IN,Base1,Ts_second


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value)   

INFO :: Matching results: Base0 - Tsupply


OUT||Base0||Tsupply
2000-01-01 01:00:00    66.106509
2000-01-01 02:00:00    52.797899
2000-01-01 03:00:00    52.736479
2000-01-01 04:00:00    52.725908
2000-01-01 05:00:00    52.743344
2000-01-01 06:00:00    52.685747
2000-01-01 07:00:00    52.690941
2000-01-01 08:00:00    52.635027
2000-01-01 09:00:00    52.638512
2000-01-01 10:00:00    52.595749
2000-01-01 11:00:00    52.607936
2000-01-01 12:00:00    52.593643
2000-01-01 13:00:00    52.546073
2000-01-01 14:00:00    52.541159
2000-01-01 15:00:00    52.489874
2000-01-01 16:00:00    52.495541
2000-01-01 17:00:00    52.481890
2000-01-01 18:00:00    52.525882
2000-01-01 19:00:00    52.539632
2000-01-01 20:00:00    52.591152
2000-01-01 21:00:00    52.610156
2000-01-01 22:00:00    52.658185
2000-01-01 23:00:00    52.675297
2000-01-02 00:00:00    52.716535
dtype: float64


In [6]:
for key, value in sim.get_results_by_pattern("IN*Base0*").items():
    print(key)
    print(value)

INFO :: Matching results: Base0 - Treturn


IN||Base0||Treturn
2000-01-01 01:00:00    26.326501
2000-01-01 02:00:00    26.326501
2000-01-01 02:00:00    26.326061
2000-01-01 03:00:00    26.326061
2000-01-01 03:00:00    26.313561
2000-01-01 04:00:00    26.313561
2000-01-01 04:00:00    26.293565
2000-01-01 05:00:00    26.293565
2000-01-01 05:00:00    26.269450
2000-01-01 06:00:00    26.269450
2000-01-01 06:00:00    26.243171
2000-01-01 07:00:00    26.243171
2000-01-01 07:00:00    26.216327
2000-01-01 08:00:00    26.216327
2000-01-01 08:00:00    26.192526
2000-01-01 09:00:00    26.192526
2000-01-01 09:00:00    26.175406
2000-01-01 10:00:00    26.175406
2000-01-01 10:00:00    26.163934
2000-01-01 11:00:00    26.163934
2000-01-01 11:00:00    26.149059
2000-01-01 12:00:00    26.149059
2000-01-01 12:00:00    26.125163
2000-01-01 13:00:00    26.125163
2000-01-01 13:00:00    26.096367
2000-01-01 14:00:00    26.096367
2000-01-01 14:00:00    26.067290
2000-01-01 15:00:00    26.067290
2000-01-01 15:00:00    26.051540
2000-01-01 16:00:00    2

In [7]:
for key, value in sim.get_results_by_pattern("IN*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - Ts_second


IN||Base1||Ts_second
2000-01-01 00:00:00    66.106509
2000-01-01 01:00:00    66.106509
2000-01-01 01:00:00    52.797899
2000-01-01 02:00:00    52.797899
2000-01-01 02:00:00    52.736479
2000-01-01 03:00:00    52.736479
2000-01-01 03:00:00    52.725908
2000-01-01 04:00:00    52.725908
2000-01-01 04:00:00    52.743344
2000-01-01 05:00:00    52.743344
2000-01-01 05:00:00    52.685747
2000-01-01 06:00:00    52.685747
2000-01-01 06:00:00    52.690941
2000-01-01 07:00:00    52.690941
2000-01-01 07:00:00    52.635027
2000-01-01 08:00:00    52.635027
2000-01-01 08:00:00    52.638512
2000-01-01 09:00:00    52.638512
2000-01-01 09:00:00    52.595749
2000-01-01 10:00:00    52.595749
2000-01-01 10:00:00    52.607936
2000-01-01 11:00:00    52.607936
2000-01-01 11:00:00    52.593643
2000-01-01 12:00:00    52.593643
2000-01-01 12:00:00    52.546073
2000-01-01 13:00:00    52.546073
2000-01-01 13:00:00    52.541159
2000-01-01 14:00:00    52.541159
2000-01-01 14:00:00    52.489874
2000-01-01 15:00:00   

In [8]:
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    print(key)
    print(value)

INFO :: Matching results: Base1 - Tr_second


OUT||Base1||Tr_second
2000-01-01 01:00:00    26.326501
2000-01-01 02:00:00    26.326061
2000-01-01 03:00:00    26.313561
2000-01-01 04:00:00    26.293565
2000-01-01 05:00:00    26.269450
2000-01-01 06:00:00    26.243171
2000-01-01 07:00:00    26.216327
2000-01-01 08:00:00    26.192526
2000-01-01 09:00:00    26.175406
2000-01-01 10:00:00    26.163934
2000-01-01 11:00:00    26.149059
2000-01-01 12:00:00    26.125163
2000-01-01 13:00:00    26.096367
2000-01-01 14:00:00    26.067290
2000-01-01 15:00:00    26.051540
2000-01-01 16:00:00    26.059765
2000-01-01 17:00:00    26.084835
2000-01-01 18:00:00    26.117148
2000-01-01 19:00:00    26.152066
2000-01-01 20:00:00    26.187031
2000-01-01 21:00:00    26.220314
2000-01-01 22:00:00    26.251229
2000-01-01 23:00:00    26.279788
2000-01-02 00:00:00    26.307007
dtype: float64
